# Data Science Essentials: Data Cleaning
    Maxwell Nielsen
    Section 001
    10/4/2022
    

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

### Problem 1

The g\_t\_results.csv file is a set of parent-reported scores on their child's Gifted and Talented tests. 
The two tests, OLSAT and NNAT, are used by NYC to determine if children are qualified for gifted programs.
The OLSAT Verbal has 16 questions for Kindergardeners and 30 questions for first, second, and third graders.
The NNAT has 48 questions. 
Using this dataset, answer the following questions.


1) What column has the highest number of null values and what percent of its values are null? Print the answer as a tuple with (column name, percentage). Make sure the second value is a percent.

2) List the columns that should be numeric that aren't. Print the answer as a tuple.

3) How many third graders have scores outside the valid range for the OLSAT Verbal Score? Print the answer

4) How many data values are missing (NaN)? Print the number.

Each part is one point.

In [6]:
df = pd.read_csv('g_t_results.csv')   #read in the data file
highest_col = df.isna().sum().idxmax()
per = (df.isna().sum().max() / len(df[highest_col]))*100   #find the column with the most nulls
print(highest_col, per, '\n')

print(('OLSAT Verbal Score', 'OLSAT Verbal Percentile', 'NNAT Non Verbal Raw Score'))

third_graders = df[df['Entering Grade Level'] == '3']
scores = (third_graders['OLSAT Verbal Score']).to_numpy()    #see which third graders have invalid scores
scores = scores.astype(int)

print('\n', len(scores[scores > 30]) + len(scores[scores<0]), '\n')

missing = df.isna().to_numpy()    #find how many missing values there are

print(len(missing[missing==1]))   

School Assigned 75.21367521367522 

('OLSAT Verbal Score', 'OLSAT Verbal Percentile', 'NNAT Non Verbal Raw Score')

 1 

192


### Problem 2

imdb.csv contains a small set of information about 99 movies. Clean the data set by doing the following in order: 

1) Remove duplicate rows by dropping the first **or** last. Print the shape of the dataframe after removing the rows.

2) Drop all rows that contain missing data. Print the shape of the dataframe after removing the rows.

3) Remove rows that have data outside valid data ranges and explain briefly how you determined your ranges for each column.

4) Identify and drop columns with three or fewer different values. Print a tuple with the names of the columns dropped.

5) Convert the titles to all lower case.

Print the first five rows of your dataframe.

In [7]:
df = pd.read_csv('imdb.csv')   #read in the data file
df.drop_duplicates(keep='first',inplace=True)    #drop all duplicate rows
print(df.shape, '\n')

df.dropna(inplace=True)      #drop all rows with null values
print(df.shape,'\n')

for ind in df.index:
    if (df.loc[ind, 'title_year']) < 1900:   #get rid of movies with wrong years
        df.drop(ind,inplace=True)

for ind in df.index:
    if (df.loc[ind, 'duration']) < 45:
        df.drop(ind,inplace=True)

for ind in df.index:
    if (df.loc[ind, 'duration']) > 300:    #get rid of movies that have infeasible times
        df.drop(ind,inplace=True)

for ind in df.index:
    if (df.loc[ind, 'imdb_score']) < 0:     #get rid of movies with unrealistic scores
        df.drop(ind,inplace=True)

print("I used the describe method to see which columns had infeasible values based upon the maxs, mins, etc.", '\n')

cols = df.columns.to_numpy()
dropped_cols = []
for col in cols:
    if len(df[col].value_counts()) <= 3:    #drop any columns with three or fewer different vals
        df.drop(col, axis=1,inplace=True)
        dropped_cols.append(col)
print(tuple(dropped_cols), '\n')

df['movie_title'] = df['movie_title'].str.lower()    #make the title column lower case

display(df.iloc[0:5])

(93, 13) 

(64, 13) 

I used the describe method to see which columns had infeasible values based upon the maxs, mins, etc. 

('color', 'language') 



,director_name,duration,gross,genres,movie_title,title_year,country,budget,imdb_score,actors,movie_facebook_likes
0,Martin Scorsese,240,116866727.0,Biography|Comedy|Crime|Drama,the wolf of wall street,2013,USA,100000000.0,8.2,"Leonardo DiCaprio,Matthew McConaughey,Jon Favreau",138000
1,Shane Black,195,408992272.0,Action|Adventure|Sci-Fi,iron man 3,2013,USA,200000000.0,7.2,"Robert Downey Jr.,Jon Favreau,Don Cheadle",95000
2,Quentin Tarantino,187,54116191.0,Crime|Drama|Mystery|Thriller|Western,the hateful eight,2015,USA,44000000.0,7.9,"Craig Stark,Jennifer Jason Leigh,Zoë Bell",114000
3,Kenneth Lonergan,186,46495.0,Drama,margaret,2011,usa,14000000.0,6.5,"Matt Damon,Kieran Culkin,John Gallagher Jr.",0
4,Peter Jackson,186,258355354.0,Adventure|Fantasy,the hobbit: the desolation of smaug,2013,USA,225000000.0,7.9,"Aidan Turner,Adam Brown,James Nesbitt",83000


### Problem 3

basketball.csv contains data for all NBA players between 2001 and 2018.
Each row represents a player's stats for a year.

Create two new features:

    career_length (int): number of years player has been playing (start at 0).
    
    target (str): The target team if the player is leaving. If the player is retiring, the target should be 'retires'.
                  A player is retiring if their name doesn't exist the next year.
                  (Set the players in 2019 to NaN).

Remove all duplicate players in each year.
Remove all rows except those where a player changes team, that is, target is not null nor 'retires'.

Drop the player, year, and team_id columns.

Return the first 10 lines of your dataframe.

In [8]:
df = pd.read_csv('basketball.csv')   #read in the data file
df['career length'] = df.apply(lambda x: x['year'] - df.loc[df['player'] == x['player'],['year']].min(), axis=1)   #calculate the career lengths of the players

df.drop_duplicates(subset=['player','year'], inplace=True)   #remove all duplicate players in each year

nextyr = [df[df['player'] == player][df['year']==year+1]['team_id'] for player, year in zip(df['player'], df['year'])]  #shift the df by a year

df['target'] = ['retires' if len(stat) == 0 else np.unique(stat)[0] for stat in nextyr] #set target to retires if their name doesn't appear next year

df.loc[df['year'] == 2019, 'target'] = np.nan       #set the target teams in 2019 to nan

df = df[df['target'] != df['team_id']]

df = df[df['year'] != 2019]

df = df[df['target'] != 'retires']    #remove all rows except for players changing teams

df.drop(['player', 'year', 'team_id'], axis = 1, inplace=True)

display(df.head(10))

/var/folders/5r/5n4nm5w14254l_1t_2lqx2nh0000gn/T/ipykernel_97111/1583357821.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  nextyr = [df[df['player'] == player][df['year']==year+1]['team_id'] for player, year in zip(df['player'], df['year'])]  #shift the df by a year


,age,per,ws,bpm,career length,target
453,27,8.2,1.0,-2.5,5,PHO
461,24,13.0,1.2,-0.9,2,ATL
462,24,15.9,6.2,2.9,3,MEM
464,33,12.7,3.7,-1.9,14,HOU
467,32,11.8,5.3,0.7,13,PHO
477,29,7.5,1.1,-2.8,9,MIN
482,31,14.1,1.9,-0.2,10,SAS
489,25,14.1,2.9,-2.4,6,CHO
490,29,12.6,2.8,0.1,2,SAC
493,28,13.0,0.0,-3.2,7,MIL


### Problem 4

Load housing.csv into a dataframe with index=0. Descriptions of the features are in housing_data_description.txt.  
The goal is to construct a regression model that predicts SalePrice using the other features of the dataset.  Do this as follows:

	1) Identify and handle the missing data.  Hint: Dropping every row with some missing data is not a good choice because it gives you an empty dataframe.  What can you do instead?
    FIXME
	2) Identify the variable with nonnumeric values that are misencoded as numbers.  One-hot encode it. Hint: don't forget to remove one of the encoded columns to prevent collinearity with the constant column (which you will add later).
    
    3) Add a constant column to the dataframe.

    4) Save a copy of the dataframe.

	5) Choose four categorical featrues that seem very important in predicting SalePrice. One-hot encode these features and remove all other categorical features.
		
	6) Run an OLS using all numerical data regression on your model.  

	
Print the ten features that have the highest coef in your model and the summary. Don't print the OLS

In [9]:
housing_dat = pd.read_csv('housing.csv', index_col=0)   #read in the data

for col in housing_dat.columns.tolist():
    housing_dat[col].fillna(housing_dat[col].mode()[0], inplace=True)    #handle the missing data

housing_dat = pd.get_dummies(housing_dat, columns=['MSSubClass'], drop_first=True)  #one-hot encode misencoded col
housing_dat['constant'] = np.ones(1460)    #add constant column

housing_copy = housing_dat.copy(deep=True)
housing_copy.to_csv('house_copy.csv')      #copy the dataframe

#choose KitchenQual, GarageQual, ExterQual, BsmtQual
dropped_cols = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType','ExterCond', 'Foundation',
        'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition']
housing_dat.drop(columns=dropped_cols, inplace=True)  #drop all the categorical columns besides what we want and encode
housing_dat = pd.get_dummies(housing_dat, columns=['KitchenQual','GarageQual','ExterQual','BsmtQual'], drop_first=True)

import statsmodels.api as sm
results = sm.OLS(housing_dat['SalePrice'], housing_dat.drop(columns='SalePrice')).fit()
# Print the summary
a = results.summary()
# Convert the summary table to a dataframe     #regress using all the numerical data and categoricals we want
results_as_html = a.tables[1].as_html()
result_df = pd.read_html(results_as_html, header=0, index_col=0)[0]

res = result_df.sort_values(['coef'], ascending=False) 
display(res.head(10))

,coef,std err,t,P>|t|,[0.025,0.975]
constant,1.327000e+06,1320000.000,1.006,0.314,-1260000.000,3910000.000
GarageCars,1.327000e+04,2720.639,4.877,0.000,7931.950,18600.000
OverallQual,1.144000e+04,1213.887,9.423,0.000,9056.723,13800.000
BsmtFullBath,8.714619e+03,2448.795,3.559,0.000,3910.912,13500.000
FullBath,6.822753e+03,2671.933,2.553,0.011,1581.325,12100.000
OverallCond,5.429463e+03,994.474,5.460,0.000,3478.642,7380.283
Fireplaces,4.890018e+03,1655.745,2.953,0.003,1642.007,8138.029
HalfBath,3.243152e+03,2564.081,1.265,0.206,-1786.706,8273.011
TotRmsAbvGrd,2.151416e+03,1171.678,1.836,0.067,-147.019,4449.851
BsmtHalfBath,1.379177e+03,3795.108,0.363,0.716,-6065.539,8823.892


### Problem 5

Using the copy of the dataframe you created in Problem 4, one-hot encode all the categorical variables.
Print the shape of the dataframe and run OLS.

Print the ten features that have the highest coef in your model and the summary.
Write a couple of sentences discussing which model is better and why.

In [10]:
new_dat = pd.read_csv('house_copy.csv')     #read in data from copied dataframe

new_dat = pd.get_dummies(new_dat, drop_first=True)   #encode all categorical variables

print(new_dat.shape)   #print the shape of the dataframe

import statsmodels.api as sm
results = sm.OLS(new_dat['SalePrice'], new_dat.drop(columns='SalePrice')).fit()
# Print the summary
a = results.summary()
# Convert the summary table to a dataframe
results_as_html = a.tables[1].as_html()
result_df = pd.read_html(results_as_html, header=0, index_col=0)[0]   #regress

res = result_df.sort_values(['coef'], ascending=False)
display(res.head(10))

print("The model in problem 4 is better. This is because the standard errors as well as the t values are smaller "
    "for the coefficients in this model. This means this model is more precise. ")

(1460, 261)


,coef,std err,t,P>|t|,[0.025,0.975]
RoofMatl_Membran,759800.0,50100.0,15.171,0.000,662000.0,858000.0
RoofMatl_Metal,729300.0,49300.0,14.785,0.000,632000.0,826000.0
RoofMatl_WdShngl,719900.0,37100.0,19.380,0.000,647000.0,793000.0
RoofMatl_Tar&Grv,665700.0,40200.0,16.549,0.000,587000.0,745000.0
RoofMatl_CompShg,663500.0,36000.0,18.454,0.000,593000.0,734000.0
RoofMatl_Roll,655400.0,44100.0,14.857,0.000,569000.0,742000.0
RoofMatl_WdShake,654400.0,39400.0,16.598,0.000,577000.0,732000.0
GarageCond_Gd,130900.0,36400.0,3.595,0.000,59500.0,202000.0
GarageCond_TA,126800.0,34800.0,3.649,0.000,58600.0,195000.0
GarageCond_Po,126700.0,38200.0,3.319,0.001,51800.0,202000.0


The model in problem 4 is better. This is because the standard errors as well as the t values are smaller for the coefficients in this model. This means this model is more precise. 
